In [9]:
#Inicializacion de spark session y creacion de nuestra instancia
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sesion3_UDF').getOrCreate()
from pyspark.sql.functions import current_timestamp,udf
from pyspark.sql.types import StructField,StructType,IntegerType,StringType,FloatType


In [37]:
#Vamos a usar el fichero de las ventas por ejemplo:
ventas = spark.read.csv('VentasNulos.csv',inferSchema=True,header=True)


In [38]:
ventas.show()

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
|  Juan|  null| null|    null|          C54|
| Pedro|     1|   30|Valencia|          R23|
| Maria|  null|  300|  Murcia|         null|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
|  null|  null| null|    null|         null|
| Jorge|     8| null|    null|          T19|
+------+------+-----+--------+-------------+



In [55]:
#Nos interesa modificar el campo Ciudad y ponerle tambien el pais, y en el caso de que no tenga ciudad, poner solo el pais:
#Definimos la fucion 
def Ciudad_Pais(ciudad):
    if ciudad==None:
        return "España"
    else:
        return ciudad + " (España)"

In [62]:
Ciudad_Pais('Elche')

'Elche (España)'

In [64]:
Ciudad_Pais(None)

'España'

In [57]:
#Ahora tendremos que crear nuestra funcion UDF:
#La creamos mediante la funcion udf, y usando una funcion lambda y le indicamos el tipo de dato que queremos que devuelva
UDF = udf(lambda x: Ciudad_Pais(x),StringType())

In [60]:
#Ahora le pasamos a nuestra select la funcion, con el campo al que queremos que la aplique
ventas.select(ventas["Nombre"],UDF(ventas["Ciudad"]).alias('Ciudad/Pais')).show()

+------+-----------------+
|Nombre|      Ciudad/Pais|
+------+-----------------+
|  Pepe|   Elche (España)|
|  Juan|           España|
| Pedro|Valencia (España)|
| Maria|  Murcia (España)|
|  Rosa|  Murcia (España)|
|   Ana|Alicante (España)|
|  null|           España|
| Jorge|           España|
+------+-----------------+



In [81]:
#Tambien podemos usarlo con la funcion withColumn
#Por ejemplo, creamos otra funcion para pasar a mayusculas los nombres:
def upperCase(str):
    if str == None:
        return None
    else:
        return str.upper()

In [82]:
mayusculaUDF = udf(lambda valor:upperCase(valor),StringType()) 

In [84]:
ventas.withColumn('Nombre En Mayusculas', mayusculaUDF(ventas["Nombre"])).show()

+------+------+-----+--------+-------------+--------------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|Nombre En Mayusculas|
+------+------+-----+--------+-------------+--------------------+
|  Pepe|     4|  200|   Elche|          X21|                PEPE|
|  Juan|  null| null|    null|          C54|                JUAN|
| Pedro|     1|   30|Valencia|          R23|               PEDRO|
| Maria|  null|  300|  Murcia|         null|               MARIA|
|  Rosa|     3|  350|  Murcia|          V55|                ROSA|
|   Ana|    10| 2300|Alicante|          B89|                 ANA|
|  null|  null| null|    null|         null|                null|
| Jorge|     8| null|    null|          T19|               JORGE|
+------+------+-----+--------+-------------+--------------------+



In [ ]:
#De esta forma agregamos una nueva columna con nuestra función personalizada

In [10]:
columns = ["ID","Nombre"]
data = [("1", "jorge lopez"),
    ("2", "belen sanchez"),
    ("3", "jose ruiz")]

df = spark.createDataFrame(data=data,schema=columns)

df.show()

+---+-------------+
| ID|       Nombre|
+---+-------------+
|  1|  jorge lopez|
|  2|belen sanchez|
|  3|    jose ruiz|
+---+-------------+



In [54]:
def Iniciales_mayusculas(valor):
    salida=""
    arr = valor.split(" ")
    for x in arr:
       salida= salida + x[0:1].upper() + x[1:len(x)] + " "
    return salida 

In [55]:
convertUDF = udf(lambda z: Iniciales_mayusculas(z),StringType())

In [56]:
df.select(df["ID"],convertUDF(df["Nombre"]).alias('Nombre')).show()

+---+--------------+
| ID|        Nombre|
+---+--------------+
|  1|  Jorge Lopez |
|  2|Belen Sanchez |
|  3|    Jose Ruiz |
+---+--------------+



In [92]:



#EJERCICIO
# Añade otra columna Apellido copiando la de Nombre y Consigue mediante UDF que en la columan nombre solo salga el nombre 
#y la apellido solo el apellido, ambos con la icial en mayuscula
#Crea un dataframe con la siguiente información:
#ID NOMBRE
#1 jorge lopez
#2 belen sanchez
#3 jose ruiz
#4 maria garcia
#5 joan bernabeu

def Iniciales_mayusculas(valor):
    salida=""
    arr = valor.split(" ")
    for x in arr:
       salida= salida + x[0:1].upper() + x[1:len(x)] + " "
    return salida 

def Separar_nombre(Nombre_completo):
    nombre_apellido = Nombre_completo.split(' ')
    return Iniciales_mayusculas(nombre_apellido[0])

def Separar_apellido(Nombre_completo):
    nombre_apellido = Nombre_completo.split(' ')
    return Iniciales_mayusculas(nombre_apellido[1])    
    

In [93]:
SepararUDF_nombre = udf(lambda nombre: Separar_nombre(nombre),StringType())
SepararUDF_apellido = udf(lambda apellido: Separar_apellido(apellido),StringType())

df2 = df.withColumn('Apellido',df['Nombre'])
df2.show()

+---+-------------+-------------+
| ID|       Nombre|     Apellido|
+---+-------------+-------------+
|  1|  jorge lopez|  jorge lopez|
|  2|belen sanchez|belen sanchez|
|  3|    jose ruiz|    jose ruiz|
+---+-------------+-------------+



In [94]:
df2 = df2.select(df2["ID"],
                 SepararUDF_nombre(df2['Nombre']).alias('Nombre'),
                 SepararUDF_apellido(df2['Apellido']).alias('Apellido')).show()

+---+------+--------+
| ID|Nombre|Apellido|
+---+------+--------+
|  1|Jorge |  Lopez |
|  2|Belen |Sanchez |
|  3| Jose |   Ruiz |
+---+------+--------+

